# Preliminary Modeling

In [33]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

from raw_to_transformed_data import get_sql_data

## Importing data

In [153]:
query_crashes = """
    SELECT *
    FROM crashes_joined;
    """
dbname = "chi-traffic-accidents"

In [154]:
df_crashes = get_sql_data(dbname, query_crashes)

In [155]:
df_crashes

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,report_type,crash_type,damage,date_police_notified,prim_contributory_cause,sec_contributory_cause,street_no,street_direction,street_name,beat_of_occurrence,num_units,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,injuries_unknown,crash_hour,crash_day_of_week,crash_month,latitude,longitude,intersection_related_i,hit_and_run_i,statements_taken_i,photos_taken_i,crash_date_est_i,private_property_i,dooring_i,work_zone_i,work_zone_type,workers_present_i,rd_no,lane_cnt,num_bikes_involved,num_pedestrians_involved,num_partially_ejected,num_extricated
0,c7fd0d0bd047159334ca0b404ed289baa04345b0f32dfc...,2021-07-29T16:20:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,"OVER $1,500",2021-07-29T16:41:00.000,NOT APPLICABLE,NOT APPLICABLE,1300,S,HALSTED ST,1232,2,NO INDICATION OF INJURY,0,0,0,0,0,2,0,16,5,7,41.865356089,-87.647011036,None,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0,0.0,0.0
1,ccb2af6700b19598020e00c0cff33a9ece97f21d60d734...,2021-07-29T16:20:00.000,30,TRAFFIC SIGNAL,UNKNOWN,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,"OVER $1,500",2021-07-29T17:00:00.000,IMPROPER LANE USAGE,UNABLE TO DETERMINE,10,E,VAN BUREN ST,113,2,NO INDICATION OF INJURY,0,0,0,0,0,2,0,16,5,7,41.877054179,-87.627288883,None,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0,0.0,0.0
2,06916bb2c61fa9c975e7f63054b12a84ce5f020d3bd188...,2021-07-29T16:15:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,$500 OR LESS,2021-07-29T16:30:00.000,UNABLE TO DETERMINE,NOT APPLICABLE,7150,W,ADDISON ST,1632,2,NO INDICATION OF INJURY,0,0,0,0,0,3,0,16,5,7,41.945403315,-87.805400524,None,Y,None,None,None,None,None,None,None,None,None,None,0.0,0.0,0.0,0.0
3,bda243a61ad5c49dd19cd45fcb1a8365ca87a6861a44e5...,2021-07-29T16:15:00.000,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,"OVER $1,500",2021-07-29T18:18:00.000,IMPROPER OVERTAKING/PASSING,IMPROPER OVERTAKING/PASSING,7452,N,WESTERN AVE,2411,2,NO INDICATION OF INJURY,0,0,0,0,0,2,0,16,5,7,42.017406801,-87.690344192,None,Y,None,None,None,None,None,None,None,None,None,None,0.0,0.0,0.0,0.0
4,3c5ab50f1aec8e4cb9ce102ed25946e73fd72a5d975536...,2021-07-29T16:14:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,"$501 - $1,500",2021-07-29T16:17:00.000,IMPROPER BACKING,NOT APPLICABLE,5129,N,BROADWAY,2033,2,NO INDICATION OF INJURY,0,0,0,0,0,2,0,16,5,7,41.975475399,-87.659697012,None,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527628,2706186558d306820406f53b416aa59b1f9a01143e642b...,2021-07-29T16:23:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,"$501 - $1,500",2021-07-29T16:44:00.000,UNABLE TO DETERMINE,IMPROPER OVERTAKING/PASSING,9539,S,STONY ISLAND AVE,431,2,NO INDICATION OF INJURY,0,0,0,0,0,4,0,16,5,7,41.721006706,-87.584827637,None,None,None,None,None,None,None,None,Non

In [151]:
drop_cols = ['crash_record_id', 'crash_date', 'report_type', 'prim_contributory_cause', 'intersection_related_i', 'hit_and_run_i', 'lane_cnt', 'has_injuries']
df_crashes = df_crashes.drop(columns=drop_cols)

df_crashes = df_crashes.rename(columns={"crash_day_of_week": "crash_day"})
df_crashes["street_direction"] = (
    df_crashes["street_direction"]
        .fillna(df_crashes["street_direction"].mode()[0]))

KeyError: "['has_injuries'] not found in axis"

In [ ]:
df_crashes.head()

In [38]:
y = df_crashes.pop("injuries_total")

In [39]:
y[:5]

0    0.0
1    1.0
2    0.0
3    1.0
4    2.0
Name: injuries_total, dtype: float64

In [40]:
X = df_crashes.copy()

In [41]:
X[:5]

,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,street_direction,num_units,crash_hour,crash_day,crash_month
0,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,S,2,3,Sunday,August
1,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,N,2,2,Sunday,August
2,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,S,2,2,Sunday,August
3,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,S,2,2,Sunday,August
4,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,W,3,2,Sunday,August


# Transforming data

In [42]:
numeric_cols = ["posted_speed_limit", "num_units", "crash_hour"]

In [43]:
category_cols = df_crashes.columns.difference(numeric_cols)

In [44]:
category_cols

Index(['alignment', 'crash_day', 'crash_month', 'device_condition',
       'first_crash_type', 'lighting_condition', 'road_defect',
       'roadway_surface_cond', 'street_direction', 'traffic_control_device',
       'trafficway_type', 'weather_condition'],
      dtype='object')

In [45]:
encoder=OneHotEncoder(drop=None, sparse=True)

In [46]:
onehot_crashes = encoder.fit_transform(X[category_cols])

In [47]:
matrix_cols = []
for col, ele in zip(category_cols, encoder.categories_):
    for e in ele:
        matrix_cols.append(col + "_" + e.lower())

In [48]:
X = pd.concat([X[numeric_cols], pd.DataFrame(onehot_crashes.toarray(), columns=matrix_cols)], axis=1)

In [49]:
X

,posted_speed_limit,num_units,crash_hour,alignment_curve on grade,alignment_curve on hillcrest,"alignment_curve, level",alignment_straight and level,alignment_straight on grade,alignment_straight on hillcrest,crash_day_friday,crash_day_monday,crash_day_saturday,crash_day_sunday,crash_day_thursday,crash_day_tuesday,crash_day_wednesday,crash_month_april,crash_month_august,crash_month_december,crash_month_february,crash_month_january,crash_month_july,crash_month_june,crash_month_march,crash_month_may,crash_month_november,crash_month_october,crash_month_september,device_condition_functioning improperly,device_condition_functioning properly,device_condition_missing,device_condition_no controls,device_condition_not functioning,device_condition_other,device_condition_unknown,device_condition_worn reflective material,first_crash_type_angle,first_crash_type_animal,first_crash_type_fixed object,first_crash_type_head on,first_crash_type_other noncollision,first_crash_type_other object,first_crash_type_overturned,first_crash_type_parked motor vehicle,first_crash_type_pedalcyclist,first_crash_type_pedestrian,first_crash_type_rear end,first_crash_type_rear to front,first_crash_type_rear to rear,first_crash_type_rear to side,first_crash_type_sideswipe opposite direction,first_crash_type_sideswipe same direction,first_crash_type_train,first_crash_type_turning,lighting_condition_darkness,"lighting_condition_darkness, lighted road",lighting_condition_dawn,lighting_condition_daylight,lighting_condition_dusk,lighting_condition_unknown,road_defect_debris on roadway,road_defect_no defects,road_defect_other,"road_defect_rut, holes",road_defect_shoulder defect,road_defect_unknown,road_defect_worn surface,roadway_surface_cond_dry,roadway_surface_cond_ice,roadway_surface_cond_other,"roadway_surface_cond_sand, mud, dirt",roadway_surface_cond_snow or slush,roadway_surface_cond_unknown,roadway_surface_cond_wet,street_direction_e,street_direction_n,street_direction_s,street_direction_w,traffic_control_device_bicycle crossing sign,traffic_control_device_delineators,traffic_control_device_flashing control signal,traffic_control_device_lane use marking,traffic_control_device_no controls,traffic_control_device_no passing,traffic_control_device_other,traffic_control_device_other railroad crossing,traffic_control_device_other reg. sign,traffic_control_device_other warning sign,traffic_control_device_pedestrian crossing sign,traffic_control_device_police/flagman,traffic_control_device_railroad crossing gate,traffic_control_device_rr crossing sign,traffic_control_device_school zone,traffic_control_device_stop sign/flasher,traffic_control_device_traffic signal,traffic_control_device_unknown,traffic_control_device_yield,trafficway_type_alley,trafficway_type_center turn lane,trafficway_type_divided - w/median (not raised),trafficway_type_divided - w/median barrier,trafficway_type_driveway,"trafficway_type_five point, or more",trafficway_type_four way,trafficway_type_l-intersection,trafficway_type_not divided,trafficway_type_not reported,trafficway_type_one-way,trafficway_type_other,trafficway_type_parking lot,trafficway_type_ramp,trafficway_type_roundabout,trafficway_type_t-intersection,trafficway_type_traffic route,trafficway_type_unknown,trafficway_type_unknown intersection type,trafficway_type_y-intersection,"weather_condition_blowing sand, soil, dirt",weather_condition_blowing snow,weather_condition_clear,weather_condition_cloudy/overcast,weather_condition_fog/smoke/haze,weather_condition_freezing rain/drizzle,weather_condition_other,weather_condition_rain,weather_condition_severe cross wind gate,weather_condition_sleet/hail,weather_condition_snow,weather_condition_unknown
0,30,2,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

## Baseline Model

In [50]:
X.shape

(526569, 129)

In [51]:
y.shape

(526569,)

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [53]:
model_dum = DummyRegressor(strategy="mean")
model_dum.fit(X_train, y_train)

DummyRegressor()

In [54]:
y_pred = model_dum.predict(X_test)

In [55]:
y_pred[:5]

array([0.18061865, 0.18061865, 0.18061865, 0.18061865, 0.18061865])

In [56]:
y_test[:5]

342410    0.0
262477    0.0
227422    0.0
514106    0.0
105362    0.0
Name: injuries_total, dtype: float64

In [57]:
np.sqrt(mean_squared_error(y_test, y_pred))

0.5491703132053445

## Linear Regression

In [58]:
import time

In [59]:
# X_train, X_test, y_train, y_test = train_test_split(df_crashes, y)

In [60]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

LinearRegression()

In [61]:
y_pred_lr = model_lr.predict(X_test)

In [62]:
np.sqrt(mean_squared_error(y_test, y_pred_lr))

0.5142709692712032

## Random Forest

In [63]:
n_est_rf = 100

In [64]:
model_rf = RandomForestRegressor(
    n_estimators=n_est_rf,
    max_features="auto")

start_time = time.time()
model_rf.fit(X_train, y_train)
print(time.time()-start_time)

766.1581768989563


In [65]:
start_time = time.time()
y_pred_rf = model_rf.predict(X_test)
print(time.time()-start_time)

5.021013975143433


In [66]:
np.sqrt(mean_squared_error(y_test, y_pred_rf))

0.5477801568325902

## GradientBoost

In [69]:
model_gb = GradientBoostingRegressor(
    learning_rate=0.1,
    n_estimators=100,
    max_depth=3,
    min_samples_leaf=1,
    min_samples_split=2)

start_time = time.time()
model_gb.fit(X_train, y_train)
print(time.time()-start_time)

118.20394802093506


In [70]:
start_time = time.time()
y_pred_gb = model_gb.predict(X_test)
print(time.time()-start_time)

0.7527539730072021


In [71]:
np.sqrt(mean_squared_error(y_test, y_pred_gb))

0.5105910306942633

## Adding pedestrian and ejection data

In [128]:
df_crashes = get_sql_data(dbname, query_crashes)

In [129]:
query_people = """
    SELECT crash_record_id, person_type, ejection
    FROM people;
    """

In [130]:
df_people = get_sql_data(dbname, query_people)

In [131]:
# df_people.loc[df_people["person_type"].isin(("DRIVER", "BICYCLE", "PASSENGER", "PEDESTRIAN")), ["crash_record_id", "person_type"]].pivot(columns="crash_record_id", values="person_type")

In [132]:
def subset_aggregate_people_df(df, agg_column, value_tuple, rename_dict=None):
    df_subset = df.loc[df[agg_column].isin(value_tuple), ["crash_record_id", agg_column]]
    df_subset["count"] = 1
    df_subset = df_subset.pivot_table(index="crash_record_id", columns=agg_column, values="count", aggfunc="sum", fill_value=0)
    if rename_dict:
        df_subset = df_subset.rename(columns=rename_dict)
    df_subset = df_subset.reset_index(drop=False)
    return df_subset

In [133]:
df_pt = subset_aggregate_people_df(
    df_people, agg_column="person_type", value_tuple=("BICYCLE", "PEDESTRIAN"), 
    rename_dict={
        "BICYCLE": "num_bikes_involved", 
        "PEDESTRIAN": "num_pedestrians_involved"})

In [134]:
df_ej = subset_aggregate_people_df(
    df_people, agg_column="ejection", value_tuple=("PARTIALLY EJECTED", "TOTALLY EJECTED", "TRAPPED/EXTRICATED"), 
    rename_dict={
        "PARTIALLY EJECTED": "num_partially_ejected", 
        "TOTALLY EJECTED": "num_partially_ejected",
        "TRAPPED/EXTRICATED": "num_extricated"})

In [147]:
temp = (
        df_crashes.loc[:, ["crash_record_id", "posted_speed_limit"]]
            .merge(df_pt, how="left", on="crash_record_id")
            .merge(df_ej, how="left", on="crash_record_id"))

In [148]:
temp = temp.drop(columns=["posted_speed_limit"])
temp = temp.fillna(0)
temp

,crash_record_id,num_bikes_involved,num_pedestrians_involved,num_partially_ejected,num_partially_ejected,num_extricated
0,7a63945a9bae12c05dee82c1962bf40f23cdd02ca655d7...,0.0,0.0,0.0,0.0,0.0
1,d207ce72f4348fa06dddcd98628d0dec81d4f195cb8d8b...,0.0,0.0,0.0,0.0,0.0
2,bba4db7cfef29230aa5d24f6b8912a2539b5d6f481a94e...,0.0,0.0,0.0,0.0,0.0
3,1e39110c6e01fd2f24e282cd610eb90d6f98a6241ab193...,0.0,0.0,0.0,0.0,0.0
4,d20d9db8a88fc227d48392b45380ae57898e5fcdcda7db...,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
526564,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,0.0,0.0,0.0,0.0,0.0
526565,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,0.0,0.0,0.0,0.0,0.0
526566,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,0.0,0.0,0.0,0.0,0.0
526567,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,0.0,0.0,0.0,0.0,0.0


In [137]:
df_crashes

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,report_type,prim_contributory_cause,street_direction,num_units,injuries_total,crash_hour,crash_day_of_week,crash_month,intersection_related_i,hit_and_run_i,lane_cnt,has_injuries,num_bikes_involved,num_pedestrians_involved,num_partially_ejected,num_partially_ejected,num_extricated
0,7a63945a9bae12c05dee82c1962bf40f23cdd02ca655d7...,2021-08-01 03:42:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,ON SCENE,DISREGARDING OTHER TRAFFIC SIGNS,S,2,0.0,3,Sunday,August,None,None,NaN,False,NaN,NaN,NaN,NaN,NaN
1,d207ce72f4348fa06dddcd98628d0dec81d4f195cb8d8b...,2021-08-01 02:41:00,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),DISREGARDING TRAFFIC SIGNALS,N,2,1.0,2,Sunday,August,Y,Y,NaN,True,NaN,NaN,NaN,NaN,NaN
2,bba4db7cfef29230aa5d24f6b8912a2539b5d6f481a94e...,2021-08-01 02:26:00,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,ON SCENE,IMPROPER TURNING/NO SIGNAL,S,2,0.0,2,Sunday,August,None,None,NaN,False,NaN,NaN,NaN,NaN,NaN
3,1e39110c6e01fd2f24e282cd610eb90d6f98a6241ab193...,2021-08-01 02:22:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,UNABLE TO DETERMINE,S,2,1.0,2,Sunday,August,N,Y,NaN,True,NaN,NaN,NaN,NaN,NaN
4,d20d9db8a88fc227d48392b45380ae57898e5fcdcda7db...,2021-08-01 02:08:00,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,IMPROPER LANE USAGE,W,3,2.0,2,Sunday,August,None,Y,NaN,True,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526564,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,2014-06-25 19:00:00,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,WET,UNKNOWN,NOT ON SCENE (DESK REPORT),IMPROPER OVERTAKING/PASSING,S,2,0.0,19,Wednesday,June,None,Y,NaN,False,NaN,NaN,NaN,NaN,NaN
526565,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,2014-02-24 19:45:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,WET,NO DEFECTS,NOT ON SCENE (DESK REPORT),UNABLE TO DETERMINE,N,2,0.0,19,Monday,February,None,None,NaN,False,NaN,NaN,NaN,NaN,NaN
526566,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,2014-01-21 07:40:00,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,FAILING TO YIELD RIGHT-OF-WAY,S,2,1.0,7,Tuesday,January,None,None,NaN,True,NaN,NaN,NaN,NaN,NaN
526567,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18 18:14:00,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),UNABLE TO DETERMINE,W,2,0.0,18,Saturday,January,None,None,NaN,False,NaN,NaN,NaN,NaN,NaN


In [75]:
df_people

,crash_record_id,person_type,ejection
0,7a63945a9bae12c05dee82c1962bf40f23cdd02ca655d7...,DRIVER,NONE
1,d207ce72f4348fa06dddcd98628d0dec81d4f195cb8d8b...,DRIVER,NONE
2,d207ce72f4348fa06dddcd98628d0dec81d4f195cb8d8b...,DRIVER,NONE
3,d207ce72f4348fa06dddcd98628d0dec81d4f195cb8d8b...,PASSENGER,NONE
4,d207ce72f4348fa06dddcd98628d0dec81d4f195cb8d8b...,PASSENGER,NONE
...,...,...,...
1166007,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,DRIVER,NONE
1166008,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,PASSENGER,NONE
1166009,a802658be15312809c771559e4f81088cfb226830792a5...,DRIVER,NONE
1166010,a802658be15312809c771559e4f81088cfb226830792a5...,DRIVER,NONE
